In [1]:
# Install packages if needed
!pip install tensorflow
!pip install imblearn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.9/644.9 MB 661.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 62.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 132.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 81.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 94.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.4/238.4 kB 9.4 MB/s eta 0:00:00


In [2]:
# imports
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np
import requests
import pickle
import io
import preprocessing as prep

In [ ]:
# path to the dataset (replace with your own path) MAKE SURE YOU CHANGE THE SLASHES
# path = "C:/Users/Asa/.cache/kagglehub/datasets/mostafaabla/garbage-classification/versions/1"

In [3]:

target_size = (150, 150)
X, y = prep.get_X_y(percent=1, target_size=target_size)

In [4]:
y[:5]

,battery,biological,brown-glass,cardboard,clothes,green-glass,metal,paper,plastic,shoes,trash,white-glass
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
X.shape

(7284, 150, 150, 3)

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [7]:
print("Train Lengths:", len(X_train), len(y_train))
print("Test Lengths:", len(X_test), len(y_test))

Train Lengths: 5463 5463
Test Lengths: 1821 1821


In [11]:
# defining the model
tf.config.optimizer.set_jit(True)
base_model = tf.keras.applications.Xception(input_shape=(150, 150, 3), include_top=False, weights='imagenet')

model = keras.Sequential()
model.add(base_model)

model.add(layers.Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(12, activation='softmax'))

model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ xception (Functional)                │ (None, 5, 5, 2048)          │      20,861,480 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_9 (Conv2D)                    │ (None, 5, 5, 512)           │       9,437,696 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_9                │ (None, 5, 5, 512)           │           2,048 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 2, 2, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 2048)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 512)                 │       1,049,088 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 12)                  │           6,156 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 31,356,468 (119.62 MB)

 Trainable params: 31,300,916 (119.40 MB)

 Non-trainable params: 55,552 (217.00 KB)

In [12]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [13]:
# Fit model
epochs = 10
model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=epochs
)

Epoch 1/10
171/171 ━━━━━━━━━━━━━━━━━━━━ 244s 1s/step - accuracy: 0.5298 - loss: 1.9955 - val_accuracy: 0.7446 - val_loss: 1.0641
Epoch 2/10
171/171 ━━━━━━━━━━━━━━━━━━━━ 225s 1s/step - accuracy: 0.8066 - loss: 0.6809 - val_accuracy: 0.8254 - val_loss: 0.6841
Epoch 3/10
171/171 ━━━━━━━━━━━━━━━━━━━━ 225s 1s/step - accuracy: 0.8606 - loss: 0.5166 - val_accuracy: 0.7963 - val_loss: 0.9557
Epoch 4/10
171/171 ━━━━━━━━━━━━━━━━━━━━ 217s 1s/step - accuracy: 0.9082 - loss: 0.3395 - val_accuracy: 0.7787 - val_loss: 1.0189
Epoch 5/10
171/171 ━━━━━━━━━━━━━━━━━━━━ 221s 1s/step - accuracy: 0.8951 - loss: 0.3937 - val_accuracy: 0.8682 - val_loss: 0.5754
Epoch 6/10
171/171 ━━━━━━━━━━━━━━━━━━━━ 215s 1s/step - accuracy: 0.9177 - loss: 0.3484 - val_accuracy: 0.6809 - val_loss: 2.7939
Epoch 7/10
171/171 ━━━━━━━━━━━━━━━━━━━━ 216s 1s/step - accuracy: 0.9296 - loss: 0.2902 - val_accuracy: 0.8501 - val_loss: 0.6859
Epoch 8/10
171/171 ━━━━━━━━━━━━━━━━━━━━ 220s 1s/step - accuracy: 0.9428 - loss: 0.2128 - val_accu

In [14]:
# Evaluate the model using the testing data
model.evaluate(X_test, y_test)

57/57 ━━━━━━━━━━━━━━━━━━━━ 14s 244ms/step - accuracy: 0.8152 - loss: 0.8271


[0.855555534362793, 0.8023064136505127]

In [15]:
# Saving the model
with open("final_model.pkl", "wb") as f:
    pickle.dump(model, f)